In [1]:
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
import joblib
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.ensemble import RandomForestRegressor


SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [2]:
no2_df = pd.read_csv('Data/final_data_for_training/no2_with_weather.csv', sep=',')
# pm_10_df.head(10)
no2_df = no2_df.drop(columns=['date', 'timestamp', 'type', 'sensor_id'])
# pm_10_df.head(10)
# pm_10_df.info()
# pm_10_df.describe()
# idx_to_remove = pm_10_df['value'].idxmax()
# # Drop the row
# pm_10_df = pm_10_df.drop(index=idx_to_remove)
target_col = 'value'

In [3]:
Y_scaler = MinMaxScaler()

no2_df[target_col] = Y_scaler.fit_transform(no2_df[[target_col]])
joblib.dump(Y_scaler, 'scalers/no2/y_scaler.pkl')

['scalers/no2/y_scaler.pkl']

In [4]:
X = no2_df.drop(columns=[target_col])
Y = no2_df[target_col]
X = no2_df.drop(columns=[target_col])
Y = no2_df[target_col]
X.columns

Index(['hour', 'year', 'month', 'day', 'temperature', 'humidity',
       'wind_speed'],
      dtype='object')

In [5]:
Y_binned = pd.qcut(Y, q=4, labels=False)

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, shuffle=True, random_state=SEED, test_size=0.3, stratify=Y_binned
)

In [6]:
assert X_train.shape[0] == Y_train.shape[0]
assert X_test.shape[0] == Y_test.shape[0]
assert X_train.shape[1] == X_test.shape[1]

In [7]:
X_scaler = MinMaxScaler()

X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.fit_transform(X_test)
joblib.dump(X_scaler, 'scalers/no2/x_scaler.pkl')

['scalers/no2/x_scaler.pkl']

In [8]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': uniform(0.01, 0.1),  
    'max_depth': [3, 6, 10],
    'subsample': uniform(0.8, 0.2), 
    'colsample_bytree': uniform(0.8, 0.2) 
}

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(random_state=42), 
    param_distributions=param_dist, 
    n_iter=100,  
    cv=3, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1, 
    random_state=42
)

# Fit the model
random_search.fit(X_train, Y_train)

# Best hyperparameters
print(f"Best Hyperparameters: {random_search.best_params_}")

# Best model
best_model = random_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# Evaluate best model
mae_best = mean_absolute_error(Y_test, y_pred_best)
rmse_best = root_mean_squared_error(Y_test, y_pred_best)
r2_best = r2_score(Y_test, y_pred_best)

print(f"Best Model MAE: {mae_best}")
print(f"Best Model RMSE: {rmse_best}")
print(f"Best Model R²: {r2_best}")

Best Hyperparameters: {'colsample_bytree': 0.9590372389537408, 'learning_rate': 0.037083225126207424, 'max_depth': 6, 'n_estimators': 200, 'subsample': 0.8501721054693323}
Best Model MAE: 0.02329210869960206
Best Model RMSE: 0.03326833551784844
Best Model R²: 0.4219755151070077


In [9]:
joblib.dump(best_model, 'models/no2/xgboost_model.pkl')

['models/no2/xgboost_model.pkl']

In [10]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, Y_train)

RandomForestRegressor(random_state=42)

In [11]:
rfr_pred = rf_model.predict(X_test)

In [12]:
print("MAE ->", mean_absolute_error(Y_test, rfr_pred))
print("RMSE ->", root_mean_squared_error(Y_test, rfr_pred)**0.5)
print("R2 ->", r2_score(Y_test, rfr_pred))

MAE -> 0.02594137276378417
RMSE -> 0.19669423898353197
R2 -> 0.2182810326172645


In [13]:
joblib.dump(rf_model, 'models/no2/random_forest_model.pkl')

['models/no2/random_forest_model.pkl']